In [1]:
%cd /home/thucth/Biometrics/insightface/recognition/arcface_torch

import pickle
from tqdm import tqdm
import onnxruntime as ort
import numpy as np  
import torch
import multiprocessing
import pandas as pd
import gc
import cv2

/home/thucth/Biometrics/insightface/recognition/arcface_torch


/home/thucth/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read data
with open("/home/thucth/Biometrics/Internal/test/enroll_si.txt", "r") as f:
    paths_enroll = f.readlines()
    paths_enroll = [x.strip().replace("/share/team/thucth/data/FaceReg/data_internal/NIST_Internal_TestSet","/home/thucth/Biometrics/Internal/test") for x in paths_enroll]


with open("/home/thucth/Biometrics/Internal/test/verif_si.txt", "r") as f:
    paths_verify = f.readlines()
    paths_verify = [x.strip().replace("/share/team/thucth/data/FaceReg/data_internal/NIST_Internal_TestSet","/home/thucth/Biometrics/Internal/test") for x in paths_verify]

print("enroll: ", paths_enroll[2])
print("verify: ", paths_verify[2])

enroll:  /home/thucth/Biometrics/Internal/test/hard/344523/A0005.jpg
verify:  /home/thucth/Biometrics/Internal/test/hard/344523/B0000.jpg


In [3]:
def extract_emb(paths_input:list, path_output_emb:str, infer):
    '''
    paths_input: paths to input images
    path_output_emb: path to output embedding in dataframe or csv file
    '''
    results = []
    for path_input in tqdm(paths_input):
        vector = infer(path_input)
        results.append(vector)
    if path_output_emb:
        pickle.dump(results, open(path_output_emb, "wb"))    
    return results
    

In [4]:
### Adaface finetune
sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 6

ort_sess_ada = ort.InferenceSession('/home/thucth/Biometrics/insightface/recognition/arcface_torch/work_dirs/finetune_10epoch_lr006/model_4.onnx', \
                                providers=['CPUExecutionProvider'], \
                                sess_options=sess_options)

def infer_ada(input_path):
    if input_path is None:
        img = np.random.randint(0, 255, size=(112, 112, 3), dtype=np.uint8)
    else:
        img = cv2.imread(input_path)
        assert img.shape[1] == 112

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1))
    img = torch.from_numpy(img).unsqueeze(0).float()
    img.div_(255).sub_(0.5).div_(0.5)
    output = ort_sess_ada.run(None, {'data': img.numpy()})[0]
    output = output/np.linalg.norm(output)
    return output

_ = extract_emb(paths_enroll, "./emb_results_ss/enroll_emb_ss.pkl", infer_ada)
_ = extract_emb(paths_verify, "./emb_results_ss/verify_emb_ss.pkl", infer_ada)

# from threading import Thread 
# # from multiprocessing import Process

# p1 = Thread(target=extract_emb, args=(paths_enroll, "./emb_results_ss/finetune4/enroll_emb_ss.pkl", infer_ada))
# p2 = Thread(target=extract_emb, args=(paths_verify, "./emb_results_ss/finetune4/verify_emb_ss.pkl", infer_ada))
# p1.start()
# p2.start()


 24%|██▍       | 1191/5000 [02:15<07:14,  8.77it/s]


KeyboardInterrupt: 

In [ ]:
enroll_embs = pickle.load(open("./emb_results_ss/adafinetune/enroll_emb_ss.pkl", "rb"))
verify_embs = pickle.load(open("./emb_results_ss/adafinetune/verify_emb_ss.pkl", "rb"))


In [ ]:
def compute(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

In [ ]:
def show_metric(true_scores, false_scores, threshold_step=1e-6):
    fars = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    thresholds = list()
    frrs = list()
    true_scores = sorted(true_scores) #from small to large
    false_scores = sorted(false_scores)

    for index in tqdm(range(len(fars))):
        far = fars[index]
        cutoff_index = np.floor(len(false_scores) * far).astype(np.int64)
        threshold = false_scores[cutoff_index] + threshold_step
        false_reject_count = np.count_nonzero(np.array(true_scores) >= threshold)
        frr = false_reject_count / (len(true_scores) + len(false_scores))
        thresholds.append(threshold)
        frrs.append(frr)

    df_title = pd.DataFrame([["FAR"], ["FRR"], ["Threshold"]])
    df_output = pd.DataFrame(np.array([fars, frrs, thresholds]))

    df_show = pd.concat([df_title, df_output], axis=1)
    return df_show


In [ ]:
import random
random.seed(1101)
num_pairs = len(enroll_embs)
true_scores = []
false_scores = []
for i in tqdm(range(num_pairs)):
    true_scores.append(compute(enroll_embs[i], verify_embs[i]))
    # for j in range(num_pairs):
    #     if i==j: continue
    for j in np.random.choice(list(range(0,i))+list(range(i+1, num_pairs)),size=1000):
        false_scores.append(compute(enroll_embs[i], verify_embs[j]))

In [ ]:
metrics = show_metric(true_scores, false_scores)
metrics.to_csv("./models/8epoch_lr008/metrics_epoch_7.csv")

## Create eval bin file

In [1]:
%cd /home/thucth/Biometrics/insightface/recognition/arcface_torch
import mxnet as mx
from mxnet import ndarray as nd
import argparse
import pickle
import sys
import os
import lfw
parser = argparse.ArgumentParser(description='Package LFW images')
# general
parser.add_argument('--data-dir', default='', help='')
parser.add_argument('--image-size', type=str, default='112,96', help='')
parser.add_argument('--output', default='', help='path to save.')
args = parser.parse_args()
lfw_dir = args.data_dir
image_size = [int(x) for x in args.image_size.split(',')]
lfw_pairs = lfw.read_pairs(os.path.join(lfw_dir, 'pairs.txt'))
lfw_paths, issame_list = lfw.get_paths(lfw_dir, lfw_pairs, 'jpg')
lfw_bins = []
#lfw_data = nd.empty((len(lfw_paths), 3, image_size[0], image_size[1]))
i = 0
for path in lfw_paths:
  with open(path, 'rb') as fin:
    _bin = fin.read()
    lfw_bins.append(_bin)
    #img = mx.image.imdecode(_bin)
    #img = nd.transpose(img, axes=(2, 0, 1))
    #lfw_data[i][:] = img
    i+=1
    if i%1000==0:
      print('loading lfw', i)

with open(args.output, 'wb') as f:
  pickle.dump((lfw_bins, issame_list), f, protocol=pickle.HIGHEST_PROTOCOL)


/home/thucth/Biometrics/insightface/recognition/arcface_torch


usage: ipykernel_launcher.py [-h] [--data-dir DATA_DIR]
                             [--image-size IMAGE_SIZE] [--output OUTPUT]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9068 --control=9061 --hb=9060 --Session.signature_scheme="hmac-sha256" --Session.key=b"cf64bb25-755e-48a7-a996-5735269a1a9c" --shell=9062 --transport="tcp" --iopub=9069 --f=/home/thucth/.local/share/jupyter/runtime/kernel-v2-20011y2A8935SoOHK.json


SystemExit: 2

/home/thucth/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
